# ETL Extraction Lab
**Name**: Iranzi Innocent  
**Student ID**: 67xxxx  

This Jupyter Notebook hurt demonstrates Full and Incremental Extraction for an ETL process using a synthetic sales dataset (`custom_data.csv`). The dataset contains 100+ records of sales transactions. The notebook includes code to generate the dataset, perform extractions, and document each step.

## Setup Instructions
1. Install Python from [python.org](https://www.python.org).
2. Install required libraries: `pip install pandas jupyter`
3. Run Jupyter: `jupyter notebook`
4. Open this notebook and run all cells.

In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Set random seed for reproducibility
random.seed(42)

# Generate 100 sales records
data = {
    'transaction_id': range(1, 101),
    'date': [(datetime(2025, 1, 1) + timedelta(days=random.randint(0, 90))).strftime('%Y-%m-%d') for _ in range(100)],
    'customer_id': [random.randint(1000, 9999) for _ in range(100)],
    'product': [random.choice(['Laptop', 'Phone', 'Tablet', 'Headphones']) for _ in range(100)],
    'amount': [round(random.uniform(50, 1000), 2) for _ in range(100)]
}

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv('custom_data.csv', index=False)
print("Dataset created and saved as 'custom_data.csv'")
print(df.head())  # Display first 5 rows

Dataset created and saved as 'custom_data.csv'
   transaction_id        date  customer_id product  amount
0               1  2025-03-23         1916   Phone   58.85
1               2  2025-01-15         4752  Laptop  640.10
2               3  2025-01-04         1525  Laptop  584.84
3               4  2025-02-05         6168  Laptop  144.98
4               5  2025-02-01         7572   Phone  560.75


# Full Extraction
This section performs a full extraction by reading all records from `custom_data.csv` into a pandas DataFrame.

In [2]:
import pandas as pd

# Perform Full Extraction
full_data = pd.read_csv('custom_data.csv')
print("Full Extraction: Loaded all records")
print(full_data.head())  # Display first 5 rows
print(f"Total records extracted: {len(full_data)}")

Full Extraction: Loaded all records
   transaction_id        date  customer_id product  amount
0               1  2025-03-23         1916   Phone   58.85
1               2  2025-01-15         4752  Laptop  640.10
2               3  2025-01-04         1525  Laptop  584.84
3               4  2025-02-05         6168  Laptop  144.98
4               5  2025-02-01         7572   Phone  560.75
Total records extracted: 100


# Incremental Extraction
This section extracts records with a `date` later than the timestamp stored in `last_extraction.txt`. If the file is empty or missing, it uses a default old date.

In [3]:
from datetime import datetime

# Read the last extraction timestamp
try:
    with open('last_extraction.txt', 'r') as file:
        last_extraction = file.read().strip()
        last_extraction_date = datetime.strptime(last_extraction, '%Y-%m-%d')
except (FileNotFoundError, ValueError):
    # Use a default old date if file is empty or missing
    last_extraction_date = datetime(2022, 1, 1)

# Perform Incremental Extraction
full_data['date'] = pd.to_datetime(full_data['date'])  # Convert date column to datetime
incremental_data = full_data[full_data['date'] > last_extraction_date]
print("Incremental Extraction: Loaded records after", last_extraction_date)
print(incremental_data.head())
print(f"Records extracted: {len(incremental_data)}")

# Update last_extraction.txt with the latest date
if not incremental_data.empty:
    latest_date = incremental_data['date'].max().strftime('%Y-%m-%d')
    with open('last_extraction.txt', 'w') as file:
        file.write(latest_date)
    print(f"Updated last_extraction.txt with {latest_date}")
else:
    print("No new records to update last_extraction.txt")

Incremental Extraction: Loaded records after 2025-04-05 00:00:00
Empty DataFrame
Columns: [transaction_id, date, customer_id, product, amount]
Index: []
Records extracted: 0
No new records to update last_extraction.txt


# Testing Incremental Extraction
This section adds new records to `custom_data.csv` with later dates and re-runs the incremental extraction to demonstrate it extracts only the new records.

In [4]:
# Simulate new data
new_data = {
    'transaction_id': range(101, 106),
    'date': [(datetime(2025, 4, 1) + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(5)],
    'customer_id': [random.randint(1000, 9999) for _ in range(5)],
    'product': [random.choice(['Laptop', 'Phone', 'Tablet', 'Headphones']) for _ in range(5)],
    'amount': [round(random.uniform(50, 1000), 2) for _ in range(5)]
}
new_df = pd.DataFrame(new_data)
new_df.to_csv('custom_data.csv', mode='a', header=False, index=False)
print("Added 5 new records to custom_data.csv")
print(new_df)

Added 5 new records to custom_data.csv
   transaction_id        date  customer_id     product  amount
0             101  2025-04-01         1452      Laptop  464.57
1             102  2025-04-02         2889      Laptop  985.02
2             103  2025-04-03         5279  Headphones  159.86
3             104  2025-04-04         3925      Tablet  904.53
4             105  2025-04-05         5349      Tablet  230.57


In [5]:
# Reload the updated dataset
full_data = pd.read_csv('custom_data.csv')

# Re-run Incremental Extraction
try:
    with open('last_extraction.txt', 'r') as file:
        last_extraction = file.read().strip()
        last_extraction_date = datetime.strptime(last_extraction, '%Y-%m-%d')
except (FileNotFoundError, ValueError):
    last_extraction_date = datetime(2020, 1, 1)

full_data['date'] = pd.to_datetime(full_data['date'])
incremental_data = full_data[full_data['date'] > last_extraction_date]
print("Incremental Extraction: Loaded records after", last_extraction_date)
print(incremental_data)
print(f"Records extracted: {len(incremental_data)}")

if not incremental_data.empty:
    latest_date = incremental_data['date'].max().strftime('%Y-%m-%d')
    with open('last_extraction.txt', 'w') as file:
        file.write(latest_date)
    print(f"Updated last_extraction.txt with {latest_date}")
else:
    print("No new records to update last_extraction.txt")

Incremental Extraction: Loaded records after 2025-04-05 00:00:00
Empty DataFrame
Columns: [transaction_id, date, customer_id, product, amount]
Index: []
Records extracted: 0
No new records to update last_extraction.txt


## Transform Full Data
This section applies transformations to the full dataset and saves the result to `transformed_full.csv`. The transformations include:
- **Cleaning**: Remove duplicates based on `transaction_id` and handle missing values.
- **Enrichment**: Add a `product_category` column to group products.
- **Structural**: Ensure `date` is in datetime format and add a `month` column.

In [6]:
# Load full dataset
full_data = pd.read_csv('custom_data.csv')

# Transformation 1: Cleaning
# Remove duplicates based on transaction_id
full_data = full_data.drop_duplicates(subset=['transaction_id'], keep='first')
# Handle missing values
if full_data.isnull().sum().any():
    full_data['amount'] = full_data['amount'].fillna(full_data['amount'].median())
    full_data['product'] = full_data['product'].fillna(full_data['product'].mode()[0])
    full_data['customer_id'] = full_data['customer_id'].fillna(full_data['customer_id'].mode()[0])
print("Cleaning: Removed duplicates and handled missing values")
print(f"Records after cleaning: {len(full_data)}")

# Transformation 2: Enrichment
# Add product_category column
product_category_map = {
    'Laptop': 'Computing',
    'Tablet': 'Computing',
    'Phone': 'Mobile',
    'Headphones': 'Accessories'
}
full_data['product_category'] = full_data['product'].map(product_category_map)
print("Enrichment: Added product_category column")

# Transformation 3: Structural
# Ensure date is datetime and add month column
full_data['date'] = pd.to_datetime(full_data['date'])
full_data['month'] = full_data['date'].dt.strftime('%Y-%m')
print("Structural: Standardized date and added month column")

# Save transformed data
full_data.to_csv('transformed_full.csv', index=False)
print("Saved transformed full data to 'transformed_full.csv'")
print(full_data.head())

Cleaning: Removed duplicates and handled missing values
Records after cleaning: 105
Enrichment: Added product_category column
Structural: Standardized date and added month column
Saved transformed full data to 'transformed_full.csv'
   transaction_id       date  customer_id product  amount product_category  \
0               1 2025-03-23         1916   Phone   58.85           Mobile   
1               2 2025-01-15         4752  Laptop  640.10        Computing   
2               3 2025-01-04         1525  Laptop  584.84        Computing   
3               4 2025-02-05         6168  Laptop  144.98        Computing   
4               5 2025-02-01         7572   Phone  560.75           Mobile   

     month  
0  2025-03  
1  2025-01  
2  2025-01  
3  2025-02  
4  2025-02  


## Transform Incremental Data
This section applies the same transformations to the incremental dataset and saves the result to `transformed_incremental.csv`.

In [7]:
# Use the incremental data from the last extraction
# Apply the same transformations

# Transformation 1: Cleaning
incremental_data = incremental_data.drop_duplicates(subset=['transaction_id'], keep='first')
if incremental_data.isnull().sum().any():
    incremental_data['amount'] = incremental_data['amount'].fillna(incremental_data['amount'].median())
    incremental_data['product'] = incremental_data['product'].fillna(incremental_data['product'].mode()[0])
    incremental_data['customer_id'] = incremental_data['customer_id'].fillna(incremental_data['customer_id'].mode()[0])
print("Cleaning: Removed duplicates and handled missing values")
print(f"Records after cleaning: {len(incremental_data)}")

# Transformation 2: Enrichment
incremental_data['product_category'] = incremental_data['product'].map(product_category_map)
print("Enrichment: Added product_category column")

# Transformation 3: Structural
incremental_data['date'] = pd.to_datetime(incremental_data['date'])
incremental_data['month'] = incremental_data['date'].dt.strftime('%Y-%m')
print("Structural: Standardized date and added month column")

# Save transformed data
incremental_data.to_csv('transformed_incremental.csv', index=False)
print("Saved transformed incremental data to 'transformed_incremental.csv'")
print(incremental_data.head())

Cleaning: Removed duplicates and handled missing values
Records after cleaning: 0
Enrichment: Added product_category column
Structural: Standardized date and added month column


Saved transformed incremental data to 'transformed_incremental.csv'
Empty DataFrame
Columns: [transaction_id, date, customer_id, product, amount, product_category, month]
Index: []


## Conclusion
This notebook successfully demonstrates:
- Full Extraction: Loading all 100+ records from `custom_data.csv`.
- Incremental Extraction: Extracting only new records based on the timestamp in `last_extraction.txt`.
- Transformations: Applying cleaning, enrichment, and structural changes to both full and incremental datasets.
- Testing: Adding new records and verifying incremental extraction and transformation.
The dataset, transformed outputs, and code are included in the repository.